In [1]:
import cv2
import torch
import os
import time
import pandas as pd
from ultralytics import YOLO

# Load the trained YOLOv11 model with best weights
model = YOLO("runs/detect/train4/weights/best.pt")  # Update with your actual model path

# Define input and output folders
input_folder = "10M CSA 200pcs"  # Folder containing test images
output_folder = "output_images_Cropped"  # Folder to save processed images
os.makedirs(output_folder, exist_ok=True)

# Define text properties
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_color = (0, 255, 0)  # Green text
thickness = 2

# Initialize tracking variables
total_objects_detected = 0
total_inference_time = 0
results_list = []

# Loop through all images in the input folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)

    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Skipping {image_name}: Unable to read image.")
        continue

    # Resize image to 640x640 before inference
    image = cv2.resize(image, (640, 640))

    # Measure inference time
    start_time = time.time()
    results = model(image)  # Perform object detection
    end_time = time.time()

    inference_time = end_time - start_time  # Time taken per image
    total_inference_time += inference_time  # Accumulate total time

    detections = results[0].boxes
    num_objects = len(detections)
    total_objects_detected += num_objects

    # Loop through detections and draw bounding boxes
    for box in detections:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
        conf = box.conf[0].item()  # Confidence score
        label = f"Rebar {conf:.2f}"  # Class label with confidence

        # Draw bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Put label above the bounding box
        cv2.putText(image, label, (x1, y1 - 10), font, 0.5, (0, 255, 0), thickness)

    # Display the number of objects detected on the image
    text_objects = f"Objects Detected: {num_objects}"
    text_inference = f"Inference Time: {inference_time:.4f}s"

    cv2.putText(image, text_objects, (50, 50), font, font_scale, font_color, thickness)
    cv2.putText(image, text_inference, (50, 80), font, font_scale, font_color, thickness)

    # Save the processed image
    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, image)
    print(f"Processed {image_name}: {num_objects} objects detected, Inference Time: {inference_time:.4f}s")

    # Save results to a list
    results_list.append([image_name, num_objects, inference_time])

# Compute average inference time
average_inference_time = total_inference_time / len(results_list) if results_list else 0

# Save results to a CSV file
csv_filename = "inference_results.csv"
df = pd.DataFrame(results_list, columns=["Image Name", "Objects Detected", "Inference Time (s)"])
df.to_csv(csv_filename, index=False)

print("\n✅ Inference completed for all images.")
print(f"🔹 Total objects detected across all images: {total_objects_detected}")
print(f"🔹 Average inference time: {average_inference_time:.4f}s")
print(f"🔹 Results saved in: {csv_filename}")



0: 512x512 200 Rebarss, 45.7ms
Speed: 2.0ms preprocess, 45.7ms inference, 2.3ms postprocess per image at shape (1, 3, 512, 512)
Processed IMG_20250103_160941.jpg: 200 objects detected, Inference Time: 1.1257s

0: 512x512 202 Rebarss, 47.3ms
Speed: 1.4ms preprocess, 47.3ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 512)
Processed IMG_20250104_152200.jpg: 202 objects detected, Inference Time: 0.0517s

0: 512x512 200 Rebarss, 43.5ms
Speed: 1.3ms preprocess, 43.5ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 512)
Processed IMG_20250103_160014.jpg: 200 objects detected, Inference Time: 0.0481s

0: 512x512 200 Rebarss, 43.9ms
Speed: 1.1ms preprocess, 43.9ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 512)
Processed IMG_20250104_150629.jpg: 200 objects detected, Inference Time: 0.0484s

0: 512x512 201 Rebarss, 49.0ms
Speed: 2.7ms preprocess, 49.0ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 512)
Processed IMG_20250104_151127.j